# 04 - Plant Diseases — Data Pull Notebook

This notebook downloads plant disease JSON from Perenual’s API and saves one file per disease_id.
It uses a single API key, polite pacing between requests, retries for temporary errors, and a safe stop when rate limits/quota are likely reached. It’s resume-friendly—existing files are skipped.

Before each time running, teammates can fetch from our github reporsitory first, then go through step 1 - 4, which means simply click "Run All" is enough.

## Step 1 - Configuration (range, output paths)



Set the API key, the disease_id range to fetch, basic rate-limit/retry settings, and the output folder.

In [11]:
API_KEY = "sk-AAWv68b047ef83b2b12091" 

# We need the plant disease from id 1 to 100
START_ID = 1
END_ID   = 100

# Rate limiting & retries
SLEEP_BETWEEN = 1     # wait (in seconds) between requests
MAX_RETRIES   = 5     # max retries for a single ID
BACKOFF_BASE  = 1.6   # exponential backoff base for retry delays

# Output directory & filename pattern
from pathlib import Path
OUT_DIR = Path("01_raw_data/04_plant_diseases")
FILENAME_PATTERN = "plant_disease_{disease_id}.json"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Key: ****{API_KEY[-6:]}, Disease ID range: {START_ID}~{END_ID}, Output dir: {OUT_DIR.resolve()}")

Key: ****b12091, Disease ID range: 1~100, Output dir: E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\04_plant_diseases


## Step 2 - Some Small Helper Functions



Minimal utilities to save JSON and build output file paths.

In [12]:
import time
import json
import requests
from typing import Optional, Dict, Any

def save_json(path: Path, data: Dict[str, Any]):
    # Ensure the parent folder exists, then write printed UTF-8 JSON
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

def build_filepath(disease_id: int) -> Path:
    # One JSON file per disease_id
    return OUT_DIR / FILENAME_PATTERN.format(disease_id=disease_id)

## Step 3 - Fetch one species (with retries & safe-stop)



Calls the pest/disease endpoint, handles 404s with a placeholder, retries on temporary errors with exponential backoff, and stops safely when repeated 429s or zero remaining quota are detected.

In [13]:
def fetch_plant_disease(disease_id: int, api_key: str) -> Optional[dict]:
    """
    Success → returns a dict
    404     → returns {"__missing__": True, "id": disease_id} (placeholder)
    None    → signals the main loop to stop (e.g., quota/IP limit, repeated 429, or x-ratelimit-remaining == 0)
    """
    base_url = "https://perenual.com/api/pest-disease-list"
    attempt = 0
    consecutive_429 = 0

    while attempt <= MAX_RETRIES:
        try:
            resp = requests.get(
                base_url,
                params={"id": disease_id, "key": api_key},
                headers={"accept": "application/json"},
                timeout=30
            )

            # check the remaining-quota hint if the server provides it
            remaining = resp.headers.get("x-ratelimit-remaining")
            if remaining is not None:
                try:
                    if int(remaining) <= 0:
                        print(f"[ID {disease_id}] x-ratelimit-remaining=0 → stopping safely (quota likely reached).")
                        return None
                except ValueError:
                    pass  # ignore if not an integer

            if resp.status_code == 200:
                return resp.json()

            if resp.status_code == 404:
                print(f"[ID {disease_id}] 404: this disease_id does not exist or has no data. Saving a placeholder.")
                return {"__missing__": True, "id": disease_id}

            if resp.status_code == 429:
                consecutive_429 += 1
                if consecutive_429 >= 3:
                    print(f"[ID {disease_id}] 429 occurred {consecutive_429} times in a row → stopping safely (rate/IP limit).")
                    return None
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {disease_id}] 429 Too Many Requests — waiting {wait:.1f}s before retrying…")
                time.sleep(wait)
                attempt += 1
                continue

            if resp.status_code in (500, 502, 503, 504):
                wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
                print(f"[ID {disease_id}] {resp.status_code} server error — waiting {wait:.1f}s then retrying…")
                time.sleep(wait)
                attempt += 1
                continue

            # Other unexpected statuses: log a short snippet for context
            print(f"[ID {disease_id}] HTTP {resp.status_code}: {resp.text[:200]}")
            return {"__error_status__": resp.status_code, "id": disease_id}

        except requests.RequestException as e:
            wait = (BACKOFF_BASE ** attempt) + 0.2 * attempt
            print(f"[ID {disease_id}] Network error ({type(e).__name__}): {e}. Waiting {wait:.1f}s then retrying…")
            time.sleep(wait)
            attempt += 1

    print(f"[ID {disease_id}] Retries exhausted. Skipping this one.")
    return {"__error_retries_exhausted__": True, "id": disease_id}

## Step 4 - Main loop (resume-friendly, skips existing files)



Iterates through the ID range, skips files that already exist, adds a little random jitter between requests, and safely stops when the fetcher returns None.

In [14]:
from tqdm.auto import tqdm
import random

downloaded = 0
skipped = 0

for disease_id in tqdm(range(START_ID, END_ID + 1), desc="Downloading Plant Diseases"):
    fp = build_filepath(disease_id)
    if fp.exists():
        skipped += 1
        continue

    data = fetch_plant_disease(disease_id, API_KEY)
    if data is None:
        print("Quota/rate-limit inferred → stopping safely.")
        break

    save_json(fp, data)
    downloaded += 1
    time.sleep(SLEEP_BETWEEN + random.uniform(0.0, 0.6))  # jitter helps avoid synchronized spikes

print(f"New downloads: {downloaded}, skipped (already exists): {skipped}. Output dir: {OUT_DIR.resolve()}")

New downloads: 0, skipped (already exists): 100. Output dir: E:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\01_raw_data\04_plant_diseases


## Step 5 (Optional) - Pack and download as a zip (for Colab)



On Google Colab, zip the output folder and download it locally. On a local machine, the file manager is simpler.

In [15]:
# import shutil
# from google.colab import files
#
# shutil.make_archive("04_plant_diseases", 'zip', "01_raw_data/04_plant_diseases")
# files.download("04_plant_diseases.zip")